In [149]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
%autoreload 2

In [151]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import getpass
from IPython.display import display
import json
import nltk
import numpy as np
import pandas as pd
import pkg_resources
import os
import random
import re
import seaborn as sns

import tensorflow as tf
from tensorflow.python.lib.io import file_io

In [152]:
from utils_export.dataset import Dataset, Model
from utils_export import utils_cloudml
from utils_export import utils_tfrecords

In [153]:
os.environ['GCS_READ_CACHE_MAX_SIZE_MB'] = '0' #Faster to access GCS file + https://github.com/tensorflow/tensorflow/issues/15530

In [154]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/msushkov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [155]:
# User inputs
PROJECT_NAME = 'wikidetox'

# Information about deployed model.
MODEL_NAMES = [
    'tf_gru_attention_msushkov_tuning1:v_20181216_worst',
    'tf_gru_attention_msushkov_tuning1:tf_gru_attention_toxicity_msushkov_tuning1_medium',
    'tf_gru_attention_msushkov_tuning1:tf_gru_attention_toxicity_msushkov_tuning1_best'
]

# Model description
TEXT_FEATURE_NAME = 'tokens' #Input text
SENTENCE_KEY = 'comment_key' #Input key
LABEL_NAME_PREDICTION_MODEL = 'frac_neg/logistic' # Output prediction

In [156]:
def tokenizer(text, lowercase=True):
  """Converts text to a list of words.

  Args:
    text: piece of text to tokenize (string).
    lowercase: whether to include lowercasing in preprocessing (boolean).
    tokenizer: Python function to tokenize the text on.

  Returns:
    A list of strings (words).
  """
  words = nltk.word_tokenize(text.decode('utf-8'))
  if lowercase:
    words = [w.lower() for w in words]
  return words

In [157]:
# User inputs
TOXICITY_PERFORMANCE_DATASET = 'gs://kaggle-model-experiments/resources/toxicity_q42017_validate.tfrecord'
TOXICITY_DATA_LABEL = 'frac_neg' #Name of the label in the performance dataset

# DECODING
decoding_input_features = {
  TEXT_FEATURE_NAME: tf.FixedLenFeature([], dtype=tf.string),
  TOXICITY_DATA_LABEL: tf.FixedLenFeature([], dtype=tf.float32)
}

def input_fn_performance_toxicity(max_n_examples=None, random_filter_keep_rate=1.0):
    res = utils_tfrecords.decode_tf_records_to_pandas(
        decoding_input_features,
        TOXICITY_PERFORMANCE_DATASET,
        max_n_examples,
        random_filter_keep_rate)
    res[TEXT_FEATURE_NAME] = list(map(tokenizer, res[TEXT_FEATURE_NAME]))
    res = res.rename(columns={
        TOXICITY_DATA_LABEL: 'label'})
    res['label'] = list(map(lambda x: bool(round(x)), list(res['label'])))
    final = res.copy(deep=True)
    return final

In [158]:
# User inputs.
model_input_spec = {
    TEXT_FEATURE_NAME: utils_tfrecords.EncodingFeatureSpec.LIST_STRING
} #library will use this automatically

model = Model(
    feature_keys_spec=model_input_spec,
    prediction_keys=LABEL_NAME_PREDICTION_MODEL,
    example_key=SENTENCE_KEY,
    model_names=MODEL_NAMES,
    project_name=PROJECT_NAME)

In [159]:
dataset_performance = Dataset(input_fn_performance_toxicity, TOXICITY_PERFORMANCE_DATASET)
random.seed(2018) # Need to set seed before loading data to be able to reload same data in the future
#dataset_performance.load_data(186997)
dataset_performance.load_data(10000)

InvalidArgumentError: Feature: tokens (data type: string) is required but could not be found.
	 [[Node: ParseSingleExample_16/ParseSingleExample = ParseSingleExample[Tdense=[DT_FLOAT, DT_STRING], dense_keys=["frac_neg", "tokens"], dense_shapes=[[], []], num_sparse=0, sparse_keys=[], sparse_types=[], _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReaderReadV2_16:1, ParseSingleExample_16/Const, ParseSingleExample_16/Const_1)]]

Caused by op 'ParseSingleExample_16/ParseSingleExample', defined at:
  File "/Users/msushkov/homebrew/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/msushkov/homebrew/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/msushkov/code/conversationai-models/model_evaluation/.pyenv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <m

In [ ]:
# Set recompute_predictions=False to save time if predictions are available.
dataset_performance.add_model_prediction_to_data(model, recompute_predictions=True)